In [1]:
#importing libraries
import json
import hashlib
import requests

In [ ]:
#load the json file
data = json.load(open('bitwarden_export_20240102182707.json'))
print(data)

In [ ]:
#dict file containing usernames and passwords
dict = {}
for item in data['items']:
    try:
        dict[item['login']['username']] = item['login']['password']
    except(KeyError):
        pass
print(dict)

In [ ]:
#checking if the password and username is leaked or not using haveibeenpwned
def check_leaked_password(username, password):
    """Check if the password has been leaked using haveibeenpwned API"""
    sha1_password = hashlib.sha1(password.encode('utf-8')).hexdigest().upper()
    prefix, suffix = sha1_password[:5], sha1_password[5:]
    url = f"https://api.pwnedpasswords.com/range/{prefix}"
    response = requests.get(url)
    if response.status_code == 200:
        for line in response.text.splitlines():
            if suffix in line:
                return f"The password for username '{username}' has been leaked. 1001"
    return f"The password for username '{username}' has not been leaked."



In [ ]:
leaked_usernames = []
for username, password in dict.items():
    check = check_leaked_password(username, password)
    if "1001" in check:
        leaked_usernames.append(username)
print(leaked_usernames)


In [ ]:
#show the usernames that have been leaked and their passwords
for username in leaked_usernames:
    print(f"{username}: {dict[username]}")

In [ ]:
#find the websites that contain the leaked usernames and passwords and the currusponding username and password
for item in data['items']:
    try:
        if item['login']['username'] in leaked_usernames:
            print(item['name'])
            print(item['login']['username'])
            print(item['login']['password'])
            print("--------------------------------------------------")
    except(KeyError):
        pass